# Multi-period Energy Systems MILP

__author__ = "Rahul Kakodkar"
__copyright__ = "Copyright 2023, Multi-parametric Optimization & Control Lab"
__credits__ = ["Rahul Kakodkar", "Efstratios N. Pistikopoulos"]
__license__ = "MIT"
__version__ = "1.0.7"
__maintainer__ = "Rahul Kakodkar"
__email__ = "cacodcar@tamu.edu"
__status__ = "Production"


This example provides the code for the publication:

Kakodkar R, Allen RC, Demirhan CD, Fu X, Pappas I, Mutlu M, Pistikopoulos EN. Multiperiod Modeling and Optimization of Hydrogen-Based Dense Energy Carrier Supply Chains. Processes. 2024 Feb 25;12(3):469.

Which can be accessed here: 
https://www.mdpi.com/2227-9717/12/3/469#app1-processes-12-00469

## Import Modules

In [ ]:

from energiapy.components import *
from energiapy.plot import *
from energiapy.aggregation import *
import pandas
from itertools import product
from matplotlib import rc
import matplotlib.pyplot as plt
from energiapy.model.solve import solve
from energiapy.utils.nsrdb_utils import fetch_nsrdb_data
from energiapy.model.formulate import formulate, Constraints, Objective
from energiapy.model.bounds import CapacityBounds
from energiapy.utils.data_utils import get_data, make_henry_price_df, remove_outliers, load_results, calculate_hourly
import numpy
from energiapy.conversion.photovoltaic import solar_power_output
from energiapy.conversion.windmill import wind_power_output


## Data Import

The following data is needed for the model

- solar and wind profiles : energiapy.fetch_nsrdb_data imports data from the NREL NSRDB database
- power demand : ERCOT for Houston
- Natural Gas prices: Henry Hub 


In [ ]:
horizon = 1

### Weather data

The fetch_nsrdb function accesses the [National Solar Radiation Database (NSRDB)](https://nsrdb.nrel.gov/) hosted by NREL on a Amazon Web Services (AWS) cloud through the h5py module To access large datasets, an API key can be requested from NREL. Instructions on how to set up the API key can be found here : https://github.com/NREL/hsds-examples/blob/master/notebooks/03_NSRDB_introduction.ipynb. Download data at any latitude longitude (globally) or state-county (because of repetition of county names) pairs within the US. Skim and fetch data which match different specifications, e.g. wind data for collection point at the highest elevation in the county. While HSDS allows you to splice datasets, the script allows you to also find means within ranges. Arrange data in a dataframe for multiscale analysis, with the temporal indices as tuples. [Can be saved as .csv/.txt/.json/.pkl]. Here, we import solar data as dni and wind data as wind speed for most populated data point in Harris county (TX) at an hourly resolution

In [ ]:
coord_houston = (29.56999969482422, -95.05999755859375)
weather_houston = pandas.read_csv('weather_houston.csv', index_col=0)
weather_houston.index = pandas.to_datetime(weather_houston.index, utc = True)


In [ ]:
solar_houston = solar_power_output(data= weather_houston, coord=coord_houston, sam = 'cecmod', module_params= 'Canadian_Solar_Inc__CS5P_220M', 
                                   inverter = 'cecinverter', inverter_params= 'ABB__MICRO_0_25_I_OUTD_US_208__208V_', 
                                   temperature_params= 'open_rack_glass_glass', aoi_model= 'no_loss', ac_model= 'sandia', spectral_model= 'no_loss')

In [ ]:
wind_houston = wind_power_output(data= weather_houston, roughness_length= 0.1, turbine_type= 'V100/1800', hub_height= 92, 
                                 wind_speed_model= 'logarithmic', density_model = 'ideal_gas', temperature_model = 'linear_gradient', 
                                 power_output_model = 'power_coefficient_curve', density_correction = True, obstacle_height = 0, 
                                 observation_height = 10)

### Natural gas prices

Natural gas prices are from the Henry price index at a daily temporal resolution.

The energia.make_henry_price_df function implicitly fills in gaps in data such as weekends and public holidays by repeating the last recorded prices For e.g. if the 4th of July will use the price for the 3rd of July a typical saturday and sunday will take the values for the last friday.

The stretch functionality stretches the values over the hourly temporal scale (8760) from a daily temporal scale (365), again through repetition.

Moreover, we can remove outliers usig the remove_outliers features in data_utils

In [ ]:
ng_price_df = pandas.concat(
    [make_henry_price_df(file_name='Henry_Hub_Natural_Gas_Spot_Price_Daily.csv', year=2016 +i, stretch= True) for i in range(5)])

### Demand data

Get [hourly power demand data](https://www.ercot.com/gridinfo/load/load_hist)  for Houston from ERCOT Coastal region

In [ ]:
ercot = pandas.read_csv('ercot.csv', index_col=0)
ercot.index = pandas.to_datetime(ercot.index, utc = True)
ercot.loc['2016-11-06 23:00:00'] = ercot.loc['2016-11-06 22:00:00'] #random missing value

### Technology cost

NREL [Annual Technology Baseline (ATB)](https://atb.nrel.gov/) is a good source for technology cost and their expected trajectories. The annualy updated list categorizes trajectories based on the appetite for research and policy push and proposes three distinct scenarios:

- Advanced
- Moderate
- Conservative

The data for technologies not covered by ATB are drawn from literature. The trajectories for these technologies (mostly conventional) are set with only modest reduction rates as these have already stagnated in terms of cost, having witness generations of utility scale application; e.g. steam methane reforming (SMR). 

In [ ]:
url = 'https://oedi-data-lake.s3.amazonaws.com/ATB/electricity/parquet/2022/ATBe.parquet'
raw_data = pandas.read_parquet(url)
raw_data = raw_data.astype(
    dtype={
        'core_metric_key': 'string',
        'core_metric_parameter': 'string',
        'core_metric_case': 'string',
        'crpyears': 'string',
        'technology': 'string',
        'technology_alias': 'string',
        'techdetail': 'string',
        'display_name': 'string',
        'scenario': 'string',
        'units': 'string'
    })
raw_data['technology'].unique()

def atb_gttr(core_metric_parameters, core_metric_case, crpyear, technology, techdetail, scenario):
    df_out = pandas.DataFrame()
    for i in core_metric_parameters:
        df = pandas.DataFrame(raw_data[
            (raw_data.core_metric_parameter == i) &
            (raw_data.core_metric_case == core_metric_case) &
            (raw_data.crpyears == str(crpyear)) &
            (raw_data.technology == technology) &
            (raw_data.techdetail == techdetail) &
            (raw_data.scenario == scenario)
        ][['value']])
        df = df.rename({'value': i}, axis='columns')
        df = df.reset_index()
        df_out = pandas.concat([df_out, df], axis='columns')
        df_out = df_out.loc[:, ~df_out.columns.duplicated()].copy()
        df_out = df_out.fillna(0)
        df_out = df_out.drop(columns=['index'])
    return df_out

We define some trajectories for technologies at different TRLs (high, medium, low) under different scenarios (Advanced, Moderate, Conservative)

In [ ]:

hig_trl_adv = [(1 - i/(31*15)) for i in range(31)]
hig_trl_mod = [(1 - i/(31*10)) for i in range(31)]
hig_trl_con = [(1 - i/(31*5)) for i in range(31)]

med_trl_adv = [(1 - i/(31*40)) for i in range(31)]
med_trl_mod = [(1 - i/(31*30)) for i in range(31)]
med_trl_con = [(1 - i/(31*20)) for i in range(31)]

low_trl_adv = [(1 - i/(31*70)) for i in range(31)]
low_trl_mod = [(1 - i/(31*50)) for i in range(31)]
low_trl_con = [(1 - i/(31*30)) for i in range(31)]

hig_trl_adv_df = pandas.DataFrame(
    data={'CAPEX': hig_trl_adv, 'Fixed O&M': hig_trl_adv, 'Variable O&M': hig_trl_adv})
hig_trl_mod_df = pandas.DataFrame(
    data={'CAPEX': hig_trl_mod, 'Fixed O&M': hig_trl_mod, 'Variable O&M': hig_trl_mod})
hig_trl_con_df = pandas.DataFrame(
    data={'CAPEX': hig_trl_con, 'Fixed O&M': hig_trl_con, 'Variable O&M': hig_trl_con})
med_trl_adv_df = pandas.DataFrame(
    data={'CAPEX': med_trl_adv, 'Fixed O&M': med_trl_adv, 'Variable O&M': med_trl_adv})
med_trl_mod_df = pandas.DataFrame(
    data={'CAPEX': med_trl_mod, 'Fixed O&M': med_trl_mod, 'Variable O&M': med_trl_mod})
med_trl_con_df = pandas.DataFrame(
    data={'CAPEX': med_trl_con, 'Fixed O&M': med_trl_con, 'Variable O&M': med_trl_con})
low_trl_adv_df = pandas.DataFrame(
    data={'CAPEX': low_trl_adv, 'Fixed O&M': low_trl_adv, 'Variable O&M': low_trl_adv})
low_trl_mod_df = pandas.DataFrame(
    data={'CAPEX': low_trl_mod, 'Fixed O&M': low_trl_mod, 'Variable O&M': low_trl_mod})
low_trl_con_df = pandas.DataFrame(
    data={'CAPEX': low_trl_con, 'Fixed O&M': low_trl_con, 'Variable O&M': low_trl_con})
constant_df = pandas.DataFrame(
    data={'CAPEX': [1]*31, 'Fixed O&M': [1]*31, 'Variable O&M': [1]*31})


We define different cost trajectories for different technologies under the three cost scenarios

In [ ]:

param_list = ['CAPEX', 'Fixed O&M', 'Variable O&M']
advanced_dict = {
    'RPV': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='ResPV', techdetail='Class5', scenario='Advanced'),
    'PV': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='UtilityPV', techdetail='Class1', scenario='Advanced'),
    'WF': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='LandbasedWind', techdetail='Class1', scenario='Advanced'),
    'LiI': atb_gttr(core_metric_parameters=param_list, core_metric_case='R&D', crpyear=30, technology='Commercial Battery Storage', techdetail='8Hr Battery Storage', scenario='Advanced'),
    'LII_discharge': constant_df,
    'PSH': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='Pumped Storage Hydropower', techdetail='NatlClass10', scenario='Advanced'),
    'PSH_discharge': constant_df,
    'ASMR': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='Nuclear', techdetail='NuclearSMR', scenario='Moderate'),
    'NPP': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='Nuclear', techdetail='Nuclear', scenario='Advanced'),
    'NGCC': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='NaturalGas_FE', techdetail='CCCCSHFrame95%', scenario='Conservative'),
    'SMR': hig_trl_adv_df,
    'SMRH': hig_trl_adv_df,
    'AqOff': hig_trl_adv_df,
    'DAC': low_trl_adv_df,
    'PEM': low_trl_adv_df,
    'H2_Comp': med_trl_adv_df,
}


moderate_dict = {
    'PV': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='UtilityPV', techdetail='Class1', scenario='Moderate'),
    'WF': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='LandbasedWind', techdetail='Class1', scenario='Moderate'),
    'LiI': atb_gttr(core_metric_parameters=param_list, core_metric_case='R&D', crpyear=30, technology='Commercial Battery Storage', techdetail='8Hr Battery Storage', scenario='Moderate'),
    'LII_discharge': constant_df,
    'PSH': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='Pumped Storage Hydropower', techdetail='NatlClass10', scenario='Moderate'),
    'PSH_discharge': constant_df,
    'ASMR': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='Nuclear', techdetail='NuclearSMR', scenario='Moderate'),
    'NPP': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='Nuclear', techdetail='Nuclear', scenario='Moderate'),
    'NGCC': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='NaturalGas_FE', techdetail='CCCCSHFrame95%', scenario='Conservative'),
    'SMR': hig_trl_mod_df,
    'SMRH': hig_trl_mod_df,
    'AqOff': hig_trl_mod_df,
    'DAC': low_trl_mod_df,
    'PEM': low_trl_mod_df,
    'H2_Comp': med_trl_mod_df,
}


conservative_dict = {
    'PV': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='UtilityPV', techdetail='Class1', scenario='Conservative'),
    'WF': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='LandbasedWind', techdetail='Class1', scenario='Conservative'),
    'LiI': atb_gttr(core_metric_parameters=param_list, core_metric_case='R&D', crpyear=30, technology='Commercial Battery Storage', techdetail='8Hr Battery Storage', scenario='Conservative'),
    'LII_discharge': constant_df,
    'PSH': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='Pumped Storage Hydropower', techdetail='NatlClass10', scenario='Conservative'),
    'PSH_discharge': constant_df,
    # no cons, adv
    'ASMR': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='Nuclear', techdetail='NuclearSMR', scenario='Moderate'),
    'NPP': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='Nuclear', techdetail='Nuclear', scenario='Conservative'),
    'NGCC': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='NaturalGas_FE', techdetail='CCCCSHFrame95%', scenario='Conservative'),
    'SMR': hig_trl_con_df,
    'SMRH': hig_trl_con_df,
    'AqOff': hig_trl_con_df,
    'DAC': low_trl_con_df,
    'PEM': low_trl_con_df,
    'H2_Comp': med_trl_con_df,
}


## Define temporal scale


The variabilities of energy systems are best captured over a discretized spatio-temporal scale. In energiapy, the first declaration is the temporal scale. 

For e.g.: Here we declare three temporal scales at different levels from right to left. The interpretation of these scales is merely symentic. Scales can be declared as the problem demands.
- 0, annual, with 1 discretization
- 1, daily with 365 discretization
- 2, hourly with 24 discretization

In essence, we are creating a temporal scale of 8760 points.

In [ ]:
scales = TemporalScale(discretization_list=[horizon, 365, 24])


## Additional Parameters

**For Convinience**

In [ ]:
bigM = 10**5
smallM = 10

**Tax credits per 45Q**

https://www.whitehouse.gov/briefing-room/statements-releases/2021/03/31/fact-sheet-the-american-jobs-plan/

In [ ]:
credit = {'DAC': {0: 31.77, 1: 31.77, 2: 31.77, 3: 31.77, 4: 31.77, 5: 50.00, 6: 50.00, 7: 50.00, 8: 50.00, 9: 50.00},
          'AqOff': {0: 20.22, 1: 20.22, 2: 20.22, 3: 20.22, 4: 20.22, 5: 35.00, 6: 35.00, 7: 35.00, 8: 35.00, 9: 35.00}
}


## Define single period model



The following processes are considered - 

- Modular nuclear reactors (ASMR)
- Proton Exchange Membrane Electrolyzers (PEM)
- Hydrogen Storage (H2_L_c/H2_L_d)
- Natural Gas Combined Cycle with 95% carbon capture (NGCC)
- Pumped Storage Hydropower (PSH/PSH_discharge)
- Lithium-ion batteries (LiI/LiI_discharge)
- Solar Photovoltaics (PV) 
- Wind Farms (WF)
- Steam Methane Reforming (SMR)
- SMR + Carbon Capture Utilization and Storage (SMRH)
- Hydrogen Fuel Cells (H2FC)
- Direct Air Capture (DAC)
- Offshore Aquifers (AQoff)

The stated processes utilize the following resources:

- Methane (CH4)
- Carbon Dioxide (CO2)
- CO2 in aquifer (CO2_AQoff)
- CO2 Captured from air (CO2_DAC)
- Vented CO2 (CO2_Vent)
- Hydrogen (H2)
- Water (H2O)
- Stored hydrogen (H2_L)
- Power stored in LiI (LiI_Power)
- Oxygen (O2)
- Power stored in PSH (PSH_Power)
- Power
- Solar
- Uranium
- Wind

We presuppose different pathways, the caveat here is that we only consider renewable power generation:

1. full: contains all technologies 
2. green: hydrogen production through electrolysis
3. blue: hydrogen production through methane reforming with carbon capture enforced
4. gray: hydrogen production through methane reforming, and carbon emission vented
5. There is an option of providing credits under any of the aforementioned scenarios

All the resources needed for the processes need to be defined along with thier characterists (refer to examples 'Small Energy Systems Example' and 'Multiscale Energy Systems MILP'). 

The location specific varying parameter data is provided through the location object. 

The single location scenario is then reduced using agglomerative hierarchial clustering (AHC) 

In [ ]:
def model(name: str, year:int, cost_dict:dict, credit: dict = None, pathway: str = None):
    
    if pathway is None:
        pathway = 'full'
    
    #*---------------- Resources---------------------------------

    Solar = Resource(name='Solar', cons_max=100, basis='MW', label='Solar Power')

    Wind = Resource(name='Wind', cons_max=100, basis='MW', label='Wind Power')

    Power = Resource(name='Power', basis='MW', label='Power generated')#, demand = True)

    Uranium = Resource(name='Uranium', cons_max=100,
                    price=42.70/(250/2), basis='kg', label='Uranium')

    H2 = Resource(name='H2', basis='tons', label='Hydrogen')

    H2_C = Resource(name='H2(C)', basis='tons', label='Hydrogen(C)', demand=True, revenue= 0.005)

    CO2_AQoff = Resource(name='CO2_AQoff', basis='tons',
                        label='Carbon dioxide - sequestered', sell=True)

    H2O = Resource(name='H2O', cons_max=1000,
                price=3.00, basis='tons', label='Water', block='Resource')

    CH4 = Resource(name='CH4', cons_max=1000, price=max(ng_price_df['CH4'])*1000, basis='tons',
                label='Natural gas', varying=[VaryingResource.DETERMINISTIC_PRICE])

    CO2 = Resource(name='CO2', basis='tons',
                label='Carbon dioxide', block='Resource')

    CO2_Vent = Resource(name='CO2_Vent', sell=True, basis='tons',
                        label='Carbon dioxide - Vented')

    O2 = Resource(name='O2', sell=True, basis='tons', label='Oxygen')

    CO2_DAC = Resource(name='CO2_DAC', basis='tons',
                    label='Carbon dioxide - captured')

    LiI_Power = Resource(name='LiI_Power', basis='MW',
                        label='Power in LiI', store_max=10000, store_min=smallM)

    PSH_Power = Resource(name='PSH_Power', basis='MW',
                        label='Power in PSH', store_max=10000, store_min=smallM)

    #*---------------- Processes---------------------------------


    PEM = Process(name='PEM', conversion={Power: -1, H2: 0.3537, O2: 2.8302, H2O: -3.1839}, capex= cost_dict['PEM']['CAPEX'][year]*1.55*10**6, 
                 fopex=0, vopex=0, cap_max=100, cap_min = 0, basis = 'tons', label = 'PEM' )

    # LiI = Process(name='LiI', storage=Power, capex=1302182, fopex=41432, vopex=2000,
    #             cap_max=100, cap_min=0, store_max=10000, label='Lithium-ion battery', basis='MW')


    LiI = Process(name='LiI', conversion={Power: -1, LiI_Power: 1}, capex= cost_dict['LiI']['CAPEX'][year]*1000, 
                 fopex=cost_dict['LiI']['Fixed O&M'][year]*1000, vopex=cost_dict['LiI']['Variable O&M'][year]*1000,
                store_max=10000, store_min=smallM, cap_max=bigM, cap_min=smallM, label='Lithium-ion battery', basis='MW', block='energy_storage')

    LiI_discharge = Process(name='LiI', conversion={Power: 1, LiI_Power: -1}, capex=0.1,
                            fopex=0.01, vopex=0, cap_max=bigM, cap_min=smallM,  label='Lithium-ion battery (d)', basis='MW')

    WF = Process(name='WF', conversion={Wind: -1, Power: 1}, capex= cost_dict['WF']['CAPEX'][year]*1000, 
                 fopex=cost_dict['WF']['Fixed O&M'][year]*1000, vopex=cost_dict['WF']['Variable O&M'][year]*1000, land = 10800/1800,
                cap_max=100, cap_min=0, label='Wind mill array', varying=[VaryingProcess.DETERMINISTIC_CAPACITY], basis='MW')

    PV = Process(name='PV', conversion={Solar: -1, Power: 1}, capex= cost_dict['PV']['CAPEX'][year]*1000, 
                 fopex=cost_dict['PV']['Fixed O&M'][year]*1000, vopex=cost_dict['PV']['Variable O&M'][year]*1000,  land = 10000/1800,
                cap_max=100, cap_min=0,  varying= [VaryingProcess.DETERMINISTIC_CAPACITY], label='Solar PV', basis='MW')

    H2_Comp = Process(name='H2_Comp', conversion={Power: -0.417, H2_C: 1, H2: -1}, capex= cost_dict['H2_Comp']['CAPEX'][year]*1.6 *
                 10**6, fopex= 0, vopex= 0, cap_max=100, cap_min=0,  label='Hydrogen Compression')

    SMRH = Process(name='SMRH', conversion={Power: -1.11, CH4: -3.76, H2O: -23.7, H2: 1, CO2_Vent: 1.03, CO2: 9.332}, capex= cost_dict['SMRH']['CAPEX'][year]*252000, 
                 fopex=cost_dict['SMRH']['Fixed O&M'][year]*945000, vopex=cost_dict['SMRH']['Variable O&M'][year]*51.5,
                cap_max=bigM, cap_min=smallM,  label='Steam methane reforming + CCUS', block='h2_prod')

    NGCC = Process(name='NGCC', conversion={Power: 1, CH4: -0.108, CO2_Vent: 0.297*0.05, CO2: 0.297 *
                0.95},capex= cost_dict['NGCC']['CAPEX'][year]*1000, 
                 fopex=cost_dict['NGCC']['Fixed O&M'][year]*1000, vopex=cost_dict['NGCC']['Variable O&M'][year]*1000, cap_max=bigM, cap_min=smallM,  label='NGCC + 95% CC', block='power_gen')


    NG = Process(name='NG', conversion={Power: 1, CH4: -0.108, CO2_Vent: 0.297},capex= 0.3*cost_dict['NGCC']['CAPEX'][year]*1000, 
                 fopex= 0.3*cost_dict['NGCC']['Fixed O&M'][year]*1000, vopex= 0.3*cost_dict['NGCC']['Variable O&M'][year]*1000, cap_max=bigM, cap_min=smallM,  label='NGCC + 95% CC', block='power_gen')


    SMR = Process(name='SMR', capex= cost_dict['SMR']['CAPEX'][year]*240000, 
                 fopex=cost_dict['SMR']['Fixed O&M'][year]*800000, vopex=cost_dict['SMR']['Variable O&M'][year]*30,  conversion={
                Power: -1.11, CH4: -3.76, H2O: -23.7, H2: 1, CO2_Vent: 9.4979}, cap_max=bigM, cap_min=0.01,  label='Steam methane reforming', block='h2_prod')


    PSH = Process(name='PSH', conversion={Power: -1, PSH_Power: 1}, capex= cost_dict['PSH']['CAPEX'][year]*1000, 
                 fopex=cost_dict['PSH']['Fixed O&M'][year]*1000, vopex=cost_dict['PSH']['Variable O&M'][year]*1000, cap_max=bigM, cap_min=smallM, store_min=smallM, store_max=1000, label='Pumped storage hydropower', basis='MW', block='energy_storage')

    PSH_discharge = Process(name='PSH_discharge', conversion={Power: 1, PSH_Power: -1},  capex=0.1,
                            fopex=0.01, vopex=0, cap_max=bigM, cap_min=smallM,  label='Pumped storage hydropower (d)', basis='MW')

    ASMR = Process(name='ASMR', conversion={Uranium: -4.17*10**(-5), H2O: -3.364, Power: 1}, capex= cost_dict['ASMR']['CAPEX'][year]*1000, 
                 fopex=cost_dict['ASMR']['Fixed O&M'][year]*1000, vopex=cost_dict['ASMR']['Variable O&M'][year]*1000, cap_max=bigM, cap_min=smallM, label='Small modular reactors (SMRs)', block='power_gen')

    AqOff = Process(name='AqOff', conversion={Power: -1.28, CO2_AQoff: 1, CO2: -1}, capex= cost_dict['AqOff']['CAPEX'][year]*552000, 
                fopex=0, vopex=cost_dict['AqOff']['Variable O&M'][year]*4.140, cap_max=bigM, cap_min=smallM,
                label='Offshore aquifer CO2 sequestration (SMR)', block='ccus')
    
    DAC = Process(name='DAC', capex= cost_dict['DAC']['CAPEX'][year]*1000, 
                fopex=cost_dict['DAC']['Fixed O&M'][year]*1000, vopex=cost_dict['DAC']['Variable O&M'][year]*1000, conversion={
                Power: -0.193, H2O: -4.048, CO2_DAC: 1}, cap_max=bigM, cap_min=smallM,  label='Direct air capture', block='ccus')

    
    if pathway == 'full':
        
        process_set = {LiI, WF, ASMR, PEM, H2_Comp, NGCC, AqOff, PSH, SMR, SMRH, PV}
    
    if pathway == 'green':
        
        process_set = {LiI, WF, ASMR, PEM, H2_Comp, PSH, PV}
    
    if pathway == 'blue':
         
        process_set = {LiI, WF, H2_Comp, NGCC, AqOff, PSH, SMRH, PV}
    
    if pathway == 'grey':
        
        process_set = {LiI, H2_Comp, NG, PSH, SMR}
    
    #*---------------- Location---------------------------------
    
    if credit is not None:
        houston= Location(name='HO', processes=process_set, capacity_factor={PV: solar_houston[:8760*horizon], WF: wind_houston[:8760*horizon]}, price_factor={
                   CH4: ng_price_df[:8760*horizon]}, scales=scales, label='Houston (conservative)', expenditure_scale_level=0, demand_scale_level=2, capacity_scale_level=2, 
                          price_scale_level=2, land_cost = 10000, credit = {AqOff: credit['AqOff'][year], DAC: credit['DAC'][year]})
    else:
        houston= Location(name='HO', processes=process_set, capacity_factor={PV: solar_houston[:8760*horizon], WF: wind_houston[:8760*horizon]}, price_factor={
                   CH4: ng_price_df[:8760*horizon]}, scales=scales, label='Houston (conservative)', expenditure_scale_level=0, demand_scale_level=2, capacity_scale_level=2, price_scale_level=2, land_cost = 10000)

    #*---------------- Scenario---------------------------------
    
    scenario = Scenario(name=name, network=houston, scales=scales,  expenditure_scale_level=0, scheduling_scale_level=2,
                    network_scale_level=0, demand_scale_level=1, purchase_scale_level=2, capacity_scale_level = 2, label='full_case', demand={houston: {H2_C: 0.2740}}, annualization_factor = 0.1)

    scenario_reduced =  reduce_scenario(scenario=scenario, periods=40, scale_level=1, method=Clustermethod.AHC, include = [IncludeAHC.CAPACITY, IncludeAHC.PRICE])

    
    return scenario_reduced



## Define multiperiod case studies

Multiperiod casetudies can be defined using the CaseStudy object which conviniently generates a multiperiod scenario from single scenarios. 

What differs between each single period scenario is primarily the cost of technology. Similarly, the conversion efficiency can also be augmented. 

The seven case studies defined are as follows:

1. Moderate cost trajectory scenario with carbon credits and all technologies available
2. Moderate cost trajectory scenario with only the green pathway available
3. Moderate cost trajectory scenario with only the grey pathway available
4. Moderate cost trajectory scenario with only the blue pathway available
5. Advanced cost trajectory scenario with all technologies available
6. Conservative cost trajectory scenario with all technologies available
7. Moderate cost trajectory scenario with all technologies available



In [ ]:
span_ = 10
moderate_credit = CaseStudy(name = 'moderate_credit', scenarios= [model(name = f'scenario{i}', year= i, cost_dict= moderate_dict, credit = credit) for i in range(span_)])
moderate_green = CaseStudy(name = 'moderate_green', scenarios= [model(name = f'scenario{i}', year= i, cost_dict= moderate_dict, pathway = 'green') for i in range(span_)])
moderate_grey = CaseStudy(name = 'moderate_grey', scenarios= [model(name = f'scenario{i}', year= i, cost_dict= moderate_dict, pathway = 'grey') for i in range(span_)])
moderate_blue = CaseStudy(name = 'moderate_blue', scenarios= [model(name = f'scenario{i}', year= i, cost_dict= moderate_dict, pathway = 'blue') for i in range(span_)])

advanced = CaseStudy(name = 'advanced', scenarios= [model(name = f'scenario{i}', year= i, cost_dict= advanced_dict) for i in range(span_)])
conservative = CaseStudy(name = 'conservative', scenarios= [model(name = f'scenario{i}', year= i, cost_dict=  conservative_dict) for i in range(span_)])
moderate = CaseStudy(name = 'moderate', scenarios= [model(name = f'scenario{i}', year= i, cost_dict= moderate_dict) for i in range(span_)])


## Formulate MIPs

MILPs can be formulated based on these scenarios.

The considered constraints are:

1. Cost: these evaluate the cash flows through the system. This includes, capital expenditure, fixed and variable operational expenditure, resource purchase cost
2. Inventory: Manages the storage of resources.
3. Production: Capacitates the production at the scheduling scale to the available network production capacities
4. Resource Balance: Arguably forms the core of the formulation by managing the resource balance in each time period in the scheduling scale
5. Network: These are binary constraints which decide whether a production or storage process is established or not
6. Demand: Ensures that the resource demand is met
7. Land: Calculates land use by processes
8. Credit: Calculates carbon credits earned by the use of processes

The objective is to minimize cost

For the full formulation refer: https://www.mdpi.com/article/10.3390/pr12030469/s1

In [ ]:
moderate_credit.formulate(constraints={Constraints.COST, Constraints.INVENTORY, Constraints.PRODUCTION,
                      Constraints.RESOURCE_BALANCE, Constraints.NETWORK, Constraints.DEMAND, Constraints.LAND, Constraints.CREDIT}, objective=Objective.COST)
advanced.formulate(constraints={Constraints.COST, Constraints.INVENTORY, Constraints.PRODUCTION,
                      Constraints.RESOURCE_BALANCE, Constraints.NETWORK, Constraints.DEMAND, Constraints.LAND}, objective=Objective.COST)
conservative.formulate(constraints={Constraints.COST, Constraints.INVENTORY, Constraints.PRODUCTION,
                      Constraints.RESOURCE_BALANCE, Constraints.NETWORK, Constraints.DEMAND, Constraints.LAND}, objective=Objective.COST)

moderate.formulate(constraints={Constraints.COST, Constraints.INVENTORY, Constraints.PRODUCTION,
                      Constraints.RESOURCE_BALANCE, Constraints.NETWORK, Constraints.DEMAND, Constraints.LAND}, objective=Objective.COST)
moderate_green.formulate(constraints={Constraints.COST, Constraints.INVENTORY, Constraints.PRODUCTION,
                      Constraints.RESOURCE_BALANCE, Constraints.NETWORK, Constraints.DEMAND, Constraints.LAND}, objective=Objective.COST)
moderate_blue.formulate(constraints={Constraints.COST, Constraints.INVENTORY, Constraints.PRODUCTION,
                      Constraints.RESOURCE_BALANCE, Constraints.NETWORK, Constraints.DEMAND, Constraints.LAND}, objective=Objective.COST)

moderate_grey.formulate(constraints={Constraints.COST, Constraints.INVENTORY, Constraints.PRODUCTION,
                      Constraints.RESOURCE_BALANCE, Constraints.NETWORK, Constraints.DEMAND, Constraints.LAND}, objective=Objective.COST)

## Solve the problems

The choice of solver here is GUROBI (https://www.gurobi.com/). This problem was solved on an academic licence. The solution can be saved as either a .txt, .json, or .pkl file. 

In [ ]:
moderate_credit.solve(solver='gurobi', print_solversteps=True, saveformat = '.pkl')
advanced.solve(solver='gurobi', print_solversteps=True, saveformat = '.pkl')
conservative.solve(solver='gurobi', print_solversteps=True, saveformat = '.pkl')
moderate.solve(solver='gurobi', print_solversteps=True, saveformat = '.pkl')
moderate_green.solve(solver='gurobi', print_solversteps=True, saveformat = '.pkl')
moderate_blue.solve(solver='gurobi', print_solversteps=True, saveformat = '.pkl')
moderate_grey.solve(solver='gurobi', print_solversteps=True, saveformat = '.pkl')



The results are discussed in the publication: https://www.mdpi.com/2227-9717/12/3/469#app1-processes-12-00469

## Cite Us

@article{kakodkar2024multiperiod,
  title={Multiperiod Modeling and Optimization of Hydrogen-Based Dense Energy Carrier Supply Chains},
  author={Kakodkar, Rahul and Allen, R Cory and Demirhan, C Doga and Fu, Xiao and Pappas, Iosif and Mutlu, Mete and Pistikopoulos, Efstratios N},
  journal={Processes},
  volume={12},
  number={3},
  pages={469},
  year={2024},
  publisher={MDPI}
}